In [1]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

In [2]:
from google.colab  import drive
drive.mount("/gdrive")
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [3]:
directory_path='/gdrive/My Drive/Yedek/Video_Dataset'

In [4]:
subset_paths={'train':directory_path+'/train','test':directory_path+'/test'}

In [5]:
load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
reconstructed_front = tf.keras.models.load_model('/gdrive/My Drive/Models_Static_v1/Frontal', options=load_locally)

In [6]:
!pip install hickle
import hickle as hkl
import random
from pathlib import Path
import os
import cv2
import matplotlib.pyplot as plt
from random import randint
import numpy as np
import matplotlib.pyplot as plt

In [7]:
def frames_from_video_file(video_path, autoencoder):
  result = []
  src = cv2.VideoCapture(str(video_path))

  for _ in range(3,20,8):
    ret, frame = src.read()
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    result.append(frame)

  src.release()
  result = np.array(result)[..., [2, 1, 0]]
  result=result/255.0
  result=autoencoder.encoder(result)
  return result

In [8]:
class FrameGenerator:
  def __init__(self, path, autoencoder, training = False):

    self.path = path
    self.training = training
    self.autoencoder = autoencoder
    self.class_names = sorted(set(p.name for p in self.path.iterdir() if p.is_dir()))
    self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))

  def get_files_and_class_names(self):
    video_paths = list(self.path.glob('*/*.avi'))
    classes = [p.parent.name for p in video_paths]
    return video_paths, classes

  def __call__(self):
    video_paths, classes = self.get_files_and_class_names()

    pairs = list(zip(video_paths, classes))

    if self.training:
      random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.autoencoder)
      label = self.class_ids_for_name[name] # Encode labels
      yield video_frames, label

In [9]:
train_path = Path(subset_paths['train'])
test_path = Path(subset_paths['test'])

In [10]:
fg = FrameGenerator(train_path, reconstructed_front, training=True)
print(fg.class_ids_for_name)

{'healthy': 0, 'sick': 1}


In [11]:
output_signature = (tf.TensorSpec(shape = (None, None, None, 32), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.float32))

In [12]:
train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_path,reconstructed_front, training=True),
                                          output_signature = output_signature)
test_ds = tf.data.Dataset.from_generator(FrameGenerator(test_path, reconstructed_front),
                                          output_signature = output_signature)

In [13]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.front = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(120,160,32)),
            tf.keras.layers.Conv2D(2, 3, strides=1, padding='same', activation='relu'),
            tf.keras.layers.MaxPooling2D(2, strides=2),
            tf.keras.layers.Conv2D(1, 3, strides=1, padding='same', activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(4,activation='sigmoid')])

    def call(self, x):
        out = self.front(x)

        return out

In [14]:
def action_model(convnet):
   model = tf.keras.Sequential()
   model.add(tf.keras.layers.TimeDistributed(convnet, input_shape=(3,120,160,32)))
   model.add(tf.keras.layers.LSTM(8,
    activation="tanh",
    recurrent_activation="sigmoid"))
   model.add(tf.keras.layers.Dense(2, activation='relu'))
   model.add(tf.keras.layers.Dropout(0.5))
   model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
   return model

In [15]:
convnet=ConvNet()
model=action_model(convnet)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-1),
              loss='binary_crossentropy', metrics=tf.keras.metrics.BinaryAccuracy(name='binary_accuracy', dtype=None, threshold=0.5))

In [16]:
train_ds = train_ds.batch(8)
test_ds = test_ds.batch(8)

train_frames, train_labels = next(iter(train_ds))
print(f'Shape of training set of frames: {train_frames.shape}')
print(f'Shape of training labels: {train_labels.shape}')

test_frames, test_labels = next(iter(test_ds))
print(f'Shape of validation set of frames: {test_frames.shape}')
print(f'Shape of validation labels: {test_labels.shape}')

Shape of training set of frames: (8, 3, 120, 160, 32)
Shape of training labels: (8,)
Shape of validation set of frames: (8, 3, 120, 160, 32)
Shape of validation labels: (8,)


In [17]:
epochs=20

In [18]:
history=model.fit( train_ds, epochs=epochs)

Epoch 1/20
22/22 [==============================] - 22s 639ms/step - loss: 0.6652 - binary_accuracy: 0.6364
Epoch 2/20
22/22 [==============================] - 11s 519ms/step - loss: 0.6618 - binary_accuracy: 0.6364
Epoch 3/20
22/22 [==============================] - 11s 484ms/step - loss: 0.6577 - binary_accuracy: 0.6364
Epoch 4/20
22/22 [==============================] - 10s 445ms/step - loss: 0.6587 - binary_accuracy: 0.6364
Epoch 5/20
22/22 [==============================] - 11s 478ms/step - loss: 0.6563 - binary_accuracy: 0.6364
Epoch 6/20
10/22 [============>.................] - ETA: 5s - loss: 0.6682 - binary_accuracy: 0.6125

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_ds ,return_dict=True)